In [ ]:
%load_ext autoreload
%load_ext line_profiler
%autoreload 2
%matplotlib qt

In [ ]:
import os
os.chdir("..")

In [49]:

import glob


In [ ]:
print(os.getcwd())

## Select Frames to Annotate 

In [50]:
#root = "C:\\Users\\eac84\\Desktop\\IRT Surgery"
root = "T:/ProjectData/DIEP FLAP/"
xvi_fns = glob.glob( root + "*")
len(xvi_fns)

62

In [ ]:
import glob
import ThemoData

In [ ]:
 data = ThermoData(fn)

In [ ]:
plt.figure()
plt.title("Q: Quit \n S:Save Frame\n N:Next Movie\n")
plt.axis("off")

In [ ]:
xvi_fns = glob.glob( root + "\\*\\*.H5")
for fn in xvi_fns[5:22]:
    data = ThermoData(fn)
    select_dataset_frames(fn, data, out_dir="./training")

## Annotate Data

In [ ]:
#from thermography_python.io import *
from Annotation.image_annotator import *
from thermography_python.base import *
import glob
import matplotlib.pyplot as plt

In [ ]:
root = "./"
fns = glob.glob(root +  "*.H5")
fns

In [ ]:

an = Annotate(fns,exclude_labeled=False, run=False)
an.image_dataset = "im_data"
an.label_dataset = "labels"
an.open_window()


In [ ]:
plt.imshow(read_H5(fns[9],"labels"))

In [ ]:
plt.imshow(read_H5(fns[9],"im_data")[30])

In [ ]:
an.model = model

In [ ]:
import h5py

In [ ]:
freq_slider.valmin

In [ ]:
an.save_label_images()

## Train Network

In [ ]:
import torch
import glob as glob
import matplotlib.pyplot as plt

In [ ]:
from retna.networks import *
from retna.training import *
from retna.data_loader import *
from retna.view import *


In [ ]:
#model = torch.load(".\\models\\flap_model.pt", map_location=torch.device('cpu'))
#model = torch.load('./models/Flap+Pref3.pt',  map_location=torch.device('cpu'))
model = torch.load('./models/Flap4channel1.pt',  map_location=torch.device('cpu'))


In [ ]:
in_channel = 1
out_channel = 2
h_chans = [8,8,8,8]

model = Retna_V1(in_channel, out_channel, h_chans)
#model = V_Net(in_channel, out_channel, h_chans).to(device)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [ ]:
loader.device

In [ ]:
read.read_H5(fns[1])

In [283]:
fns = np.array(xvi_fns)[[18,24,39,46]]
DataSet = H5_Handler(fns, datasets=["im_data","im_data"], which="all")
Loader = DataLoader(DataSet)
Loader.dataset.outsize = [150,200]
Loader.dataset.scale_range = [.1,.4]
Loader.dataset.rand_flip = [0.5,0]
Loader.dataset.noise = True
Loader.dataset.noise_range = [-0.2,0.2]
Loader.dataset.noise_low  = -0.3
Loader.dataset.noise_high = 0.3
Loader.dataset.rand_noise = 0.5

In [284]:
plt.close("")
print_mosaic(Loader,m=6, n=5)

s
[34]	s
[127]	s
[228]	s
[117]	s
[283]	s
[51]	s
[143]	s
[204]	s
[15]	s
[72]	s
[261]	s
[220]	s
[84]	s
[168]	s
[170]	s
[123]	s
[265]	s
[37]	s
[9]	s
[233]	s
[272]	s
[243]	s
[194]	s
[17]	s
[280]	s
[7]	s
[290]	s
[198]	s
[135]	s
[192]	

In [254]:
plt.imshow(random_ramp((20,20)))

In [239]:
def random_ramp(shape):
    a,b = np.random.randint(low=-1, high = 2, size =2)
    x = np.linspace(a,-a, shape[0])
    y = np.linspace(b,-b, shape[1])

    X, Y = np.meshgrid(x, y)

    return X*Y

In [56]:
###########################
Loader.dataset.channels[1] = None
Loader.dataset.threshold = None
Loader.dataset.outsize = [200,200]
Loader.dataset.expand_label = True
Loader.dataset.scale_range = [.3,.4]
#Loader.dataset.remap_label = [[1,0,0,0],
                              [1,1,0,0],
                              [1,0,1,0],
                              [1,0,0,1] ]

In [61]:
print_mosaic(Loader,m=5, n=5)

[86]	[17]	[27]	[88]	[64]	[50]	[63]	[38]	[11]	[108]	[93]	[38]	[23]	[97]	[92]	[16]	[42]	[67]	[73]	[11]	[73]	[7]	[110]	[74]	[47]	

In [54]:
model.train_model(Loader, num_epochs = 50)

NameError: name 'model' is not defined

[2686]	[1947]	[5390]	[4556]	[4930]	[5158]	[7008]	[4346]	[8021]	[2069]	[1443]	[6261]	[7964]	[1580]	[6243]	[1599]	[4634]	[858]	[3951]	[188]	[207]	[1557]	[2013]	[6274]	[4548]	[3752]	[5051]	[1824]	[249]	[4053]	

In [ ]:
Loader.dataset.remap_label 

In [ ]:
model

## Load the Data

In [ ]:
label_fn = glob.glob("./Training/*_label.tiff")
dataSet = Tiff_Handler(label_fn)
Loader = DataLoader(dataSet)

In [ ]:
root = "C:/Users/eac84/Desktop/Test_Flap_data/"
fns = glob.glob(root +  "2*.H5")
print(fns)
DataSet = H5_Handler(fns, datasets=["im_data","labels"], which="all")
Loader = DataLoader(DataSet)


In [ ]:
if 0:
    Loader.dataset.device="cpu"
    model = model.to("cpu")
else:
    Loader.dataset.device="cuda:0"
    model = model.to("cuda:0")

In [ ]:
in_channel = 1
out_channel = 4
h_chans = [16,16,16,16,16]
model = Retna_V1(in_channel, out_channel, h_chans)

[85]	[45]	[55]	[14]	[36]	[41]	[57]	[28]	[0]	[96]	[2]	[20]	[40]	[16]	[53]	[43]	[21]	[68]	[58]	[71]	[93]	[79]	[0]	[3]	[13]	

In [ ]:
print_comparisons(model,Loader)

In [ ]:
plt.hist(Loader.dataset.selection_weights)

In [ ]:
draw_model_params(model)

In [ ]:
Loader.dataset.reset_weights()

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = torch.load(".\\models\\flap_model.pt", map_location=torch.device('cpu'))
model  = model.to(device)

In [ ]:
#path = "./models"
#if not os.path.isdir(path): os.mkdir(path)
torch.save(model, './models/Flap4channel.pt')

In [ ]:
model

In [ ]:
model = add_block(model)

In [ ]:
Loader.dataset.reset_weights()

## Sandbox

In [ ]:

## Not sure how to add this to annotator 
fns = glob.glob(".\\training\\*_input.tiff")
cv2.namedWindow("", cv2.WINDOW_NORMAL)
cv2.resizeWindow("", 1400, 600)   

bad_names = []
for fn in fns:
    fn2 = fn.replace("input","label")
    if os.path.exists(fn2):
        im = io.imread(fn)
        im2 = io.imread(fn2)       
        draw = np.hstack((im,im2*100))
        cv2.imshow("",draw)   
        
        key = cv2.waitKeyEx(10000)
        if key == ord('s'):                
            print(".")
        
        if key == ord('b'):                
            bad_names.append( fn2 )

cv2.destroyAllWindows()   